<a href="https://colab.research.google.com/github/vmcarranzam/Proyecto_RL_CDP/blob/main/Regresi%C3%B3n_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Regresión Lineal con Python

##Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from google.colab import drive
drive.mount('/content/drive')

##Carga de datos

In [ ]:
data=np.load("/content/drive/MyDrive/Colab Notebooks/Proyecto Ciencia de Datos/proyecto_training_data.npy")
df=pd.DataFrame(data, columns=["SalePrice","OverallQual","FirstFlrSF", "TotRmsAbvGrd","YearBuilt","LotFrontage"])
df

##Definición de la clase

In [ ]:
class RegresionLineal:
    
    def __init__(self, x_col, y_col, epochs, learning_rate, imprimir_cada, df):
        self.x_col = x_col
        self.y_col = y_col
        self.x = np.array(df[x_col])
        self.y = np.array(df[y_col])
        self.epochs = epochs
        self.lr = learning_rate
        self.imprimir_cada = imprimir_cada
        self.train_x, self.test_x, self.train_y, self.test_y = self.split_data()
        self.mse = {}

 #Método para dividir el dataset   
    def split_data(self):
        idx = int(len(self.x) * 0.8)
        train_x = self.x[:idx]
        test_x = self.x[idx:]
        train_y = self.y[:idx]
        test_y = self.y[idx:]
        return train_x, test_x, train_y, test_y